In [25]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from torch_geometric.data import Data,Dataset

total_parms_new = pd.read_csv('/mnt/ssd1/sunyifan/WorkStation/dpuf/my_DPGCN/total_params_new.csv',index_col=0)
total_parms = pd.read_csv('/mnt/ssd1/sunyifan/WorkStation/dpuf/my_DPGCN/total_params.csv',index_col=0)

from typing import List, Literal, Sequence, Union


dataset_list = ['cora','citeseer','pubmed','computers','photo','cs','physics','reddit','github','flickr','lastfmasia']

def get_datasetwise_df(df:pd.DataFrame,name=None):
    if not name:
        dflist = []
        for i in ['cora','citeseer','pubmed','computers','photo','cs','physics','reddit','github','flickr','lastfmasia']:
            dflist.append(df[df['Dataset']==i])
        return dflist
    else:
        return df[df['Dataset']==name]

def baseline(df:pd.DataFrame):
    clean_adam = df[(df['Dp']==False)&(df['Rdp']==True)&(df['Ldp']==False)&(df['Optim type']=='adam')]
    clean_sgd = df[(df['Dp']==False)&(df['Rdp']==True)&(df['Ldp']==False)&(df['Optim type']=='sgd')]
    return clean_adam,clean_sgd

def rdp_drop(df:pd.DataFrame):
    labels = ['Vanilla train acc','Vanilla train loss','Vanilla train f1',
          'Shadow train acc', 'Shadow train loss',
       'Shadow train f1', 'Shadow test acc', 'Shadow test loss',
       'Shadow test f1', 'MIA subsample rate','MIA seed','Shadow runtime per', 'Num val', 'Num test', 'Layers', 'Hidden dims','Activation','Shadow learning rate','Dropout','Sampler', 'Sampler batchsize','Dp','Rdp','Ldp']
    return df.drop(columns=labels)

def get_res_rdp(cora:pd.DataFrame):
    maxx = ['Vanilla test acc','Vanilla test f1']
    meann = ['Vanilla runtime per','MIA mlp', 'MIA svm','MIA ranfor', 'MIA logi', 'MIA ada', 'MIA confidence mse','Epsilon','Delta']
    minn = ['Epsilon','Delta']
    res = {}
   
    for k in maxx:
        res[k] = cora.max()[k]
    for k in meann:
        res[k] = cora.mean(numeric_only=True)[k]
    for k in minn:
        res[k] = cora.min()[k]
    return res
    

total_parms_new.columns

Index(['Dataset', 'Vanilla train acc', 'Vanilla train loss',
       'Vanilla train f1', 'Vanilla test acc', 'Vanilla test loss',
       'Vanilla test f1', 'Shadow train acc', 'Shadow train loss',
       'Shadow train f1', 'Shadow test acc', 'Shadow test loss',
       'Shadow test f1', 'MIA subsample rate', 'MIA mlp', 'MIA svm',
       'MIA ranfor', 'MIA logi', 'MIA ada', 'MIA confidence mse',
       'MIA confidence thr', 'MIA seed', 'Vanilla runtime per',
       'Shadow runtime per', 'Epsilon', 'Delta', 'Dp', 'Rdp', 'Ldp',
       'Norm bound', 'Noise scale', 'Sampler', 'Sampler batchsize',
       'Occurance k', 'Cluster numparts', 'Saint rootnodes',
       'Saint samplecoverage', 'Saint walklenth', 'Epochs', 'Shadow epochs',
       'Num val', 'Num test', 'Layers', 'Hidden dims', 'Learning rate',
       'Shadow learning rate', 'Dropout', 'Activation', 'Early stopping',
       'Patience', 'Optim type'],
      dtype='object')

'cora','citeseer','pubmed','computers','photo','cs','physics','reddit','github','flickr','lastfmasia'

In [28]:
dfs = get_datasetwise_df(total_parms)
cora = dfs[0]
cora_adam,cora_sgd = baseline(cora)

Cluster

In [27]:
cora_adam['Sampler'].drop_duplicates()

50         cluster
270       saint_rw
530     saint_node
930       neighbor
1000     occurance
Name: Sampler, dtype: object